In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Sample data
X = np.random.rand(1000, 10, 1)  # 1000 samples, 10 timesteps, 1 feature
y = np.random.randint(0, 2, 1000)  # Binary classification (0 or 1)

# Convert labels to categorical one-hot encoding
y = to_categorical(y, num_classes=2)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the LSTM model
model = Sequential()
model.add(Input(shape=(10, 1)))  # Input layer with input shape of 10 timesteps and 1 feature
model.add(LSTM(128, activation = 'relu')) # 128 LSTM units, input shape corresponds to timesteps and features
model.add(Dense(2, activation='softmax'))  # Output layer with 2 units (for binary classification) and softmax activation

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val), verbose=0)

# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')


Validation Loss: 0.6923662424087524
Validation Accuracy: 0.47999998927116394


In [2]:
import numpy as np
from keras.models import Sequential # type: ignore
from keras.layers import LSTM, Dense # type: ignore
from sklearn.preprocessing import MinMaxScaler

# Function to generate Fibonacci sequence
def generate_fibonacci(n):
    fib_sequence = [0, 1]
    for i in range(2, n):
        fib_sequence.append(fib_sequence[-1] + fib_sequence[-2])
    return fib_sequence

# Generate Fibonacci sequence
n = 20
sequence = generate_fibonacci(n)
print("Fibonacci sequence:", sequence)

# Prepare the data
def create_dataset(seq):
    X, y = [], []
    for i in range(len(seq) - 2):
        X.append([seq[i], seq[i + 1]])
        y.append(seq[i + 2])
    return np.array(X), np.array(y)

X, y = create_dataset(sequence)

# Reshape from [samples, timesteps] into [samples, timesteps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))

print("X shape:", X.shape)
print("y shape:", y.shape)

# Define the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X.shape[1], 1)))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X, y, epochs=300, verbose=0)

# Demonstrate prediction
x_input = np.array([sequence[-2], sequence[-1]]).reshape((1, 2, 1))
yhat = model.predict(x_input, verbose=0)
print("Predicted next Fibonacci number:", yhat[0][0])


Fibonacci sequence: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]
X shape: (18, 2, 1)
y shape: (18,)


d:\09_Project_2024\04_DeepLearning\DeepLearning\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Predicted next Fibonacci number: 6763.847


In [3]:
import numpy as np
from keras.models import Sequential  # type: ignore
from keras.layers import LSTM, Dense # type: ignore
import pandas as pd

# Generate Fibonacci sequence
def generate_fibonacci(n):
    fib_sequence = [0, 1]
    for i in range(2, n):
        next_value = fib_sequence[i-1] + fib_sequence[i-2]
        fib_sequence.append(next_value)
    return fib_sequence

# Prepare the dataset
def create_dataset(sequence):
    X, y = [], []
    for i in range(len(sequence) - 2):
        X.append([sequence[i], sequence[i+1]])
        y.append(sequence[i+2])
    return np.array(X), np.array(y)

# Parameters
n_seq = 20  # Length of the Fibonacci sequence

# Generate the Fibonacci sequence
fib_sequence = generate_fibonacci(n_seq + 2)  # Generate enough values for input-output pairs

# Create the dataset
X, y = create_dataset(fib_sequence)

# Reshape from [samples, timesteps] to [samples, timesteps, features]
X = X.reshape((X.shape[0], 2, 1))

# Define the LSTM model
model = Sequential()
model.add(Input(shape=(2, 1)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Fit the model
model.fit(X, y, epochs=300, verbose=0)

# Demonstrate prediction
yhat = model.predict(X, verbose=0)
df = pd.DataFrame({'X_{t-2}': X[:, 0, 0], 'X_{t-1}': X[:, 1, 0],'y': y, 'y_hat': np.round(yhat[:, 0])})
